In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
from tqdm import tqdm
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [2]:
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', len(df))

In [3]:
df = pd.read_csv(r'new_house.csv')

In [4]:
df.head()

,links,location,name,bedrooms,bathroom,parking,furniture,sqft,agent,gym,transaction_type,price
0,https://www.magicbricks.com/propertyDetails/5-BHK-3100-Sq-ft-Penthouse-FOR-Sale-Bandra-West-in-Mumbai&id=4d423638323831343837,"Park View, Bandra West, Mumbai South West, Maharashtra, Bandra West, Mumbai - Western Mumbai, Maharashtra",5-BHK-3100-Sq-ft-Penthouse-FOR-Sale-Bandra-West-in-Mumbai,5beds,6baths,4 covered parking,NaN,3100sqft,certified agent,yes,New Property,₹19.0 cr
1,https://www.magicbricks.com/propertyDetails/5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai&id=4d423639323236333639,"Piramal Revanta, Behind Nirmal Lifestyle mall, Goregaon Mulund Link Road, Mulund West, Mulund West, Mumbai - Central Mumbai, Maharashtra",5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai,5beds,5baths,3 covered parking,unfurnished,2200sqft,Not certified,yes,New Property,₹5.90 cr
2,https://www.magicbricks.com/propertyDetails/4-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Thakur-Village-in-Mumbai&id=4d423637303333353637,"2602 challenger tower, Thakur Village, Mumbai - Western Mumbai, Maharashtra",4-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Thakur-Village-in-Mumbai,4beds,4baths,1 covered parking,NaN,3600sqft,Not certified,yes,Resale,₹12.0 cr
3,https://www.magicbricks.com/propertyDetails/5-BHK-6223-Sq-ft-Penthouse-FOR-Sale-Goregaon-East-in-Mumbai-r1&id=4d423531333232303132,"Goregaon East, Mumbai, Goregaon East, Mumbai - Western Mumbai, Maharashtra",5-BHK-6223-Sq-ft-Penthouse-FOR-Sale-Goregaon-East-in-Mumbai-r1,5beds,6baths,5 covered parking,NaN,4114sqft,Not certified,yes,Resale,₹22.0 cr
4,https://www.magicbricks.com/propertyDetails/4-BHK-2800-Sq-ft-Penthouse-FOR-Sale-Akurli-Nagar-in-Mumbai&id=4d423635363938323639,"Kandivali East, Mumbai, Akurli Nagar, Mumbai - Western Mumbai, Maharashtra",4-BHK-2800-Sq-ft-Penthouse-FOR-Sale-Akurli-Nagar-in-Mumbai,4beds,4baths,3 covered parking,NaN,1602sqft,certified agent,yes,New Property,₹4.95 cr


In [5]:
links = df['links'].tolist()

In [6]:
links

['https://www.magicbricks.com/propertyDetails/5-BHK-3100-Sq-ft-Penthouse-FOR-Sale-Bandra-West-in-Mumbai&id=4d423638323831343837',
 'https://www.magicbricks.com/propertyDetails/5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai&id=4d423639323236333639',
 'https://www.magicbricks.com/propertyDetails/4-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Thakur-Village-in-Mumbai&id=4d423637303333353637',
 'https://www.magicbricks.com/propertyDetails/5-BHK-6223-Sq-ft-Penthouse-FOR-Sale-Goregaon-East-in-Mumbai-r1&id=4d423531333232303132',
 'https://www.magicbricks.com/propertyDetails/4-BHK-2800-Sq-ft-Penthouse-FOR-Sale-Akurli-Nagar-in-Mumbai&id=4d423635363938323639',
 'https://www.magicbricks.com/propertyDetails/5-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Worli-Sea-Face-in-Mumbai&id=4d423638313737343231',
 'https://www.magicbricks.com/propertyDetails/3-BHK-4500-Sq-ft-Penthouse-FOR-Sale-Amboli-in-Mumbai&id=4d423636363035383037',
 'https://www.magicbricks.com/propertyDetails/4-BHK-3926-Sq-ft-Penthouse-FOR-Sale-B

In [12]:
trial = np.random.choice( links , 50)

In [43]:
d={'links':[],'location':[],'pincode':[],'name':[],'bedrooms':[],'bathroom':[],'parking':[],'furniture':[],'sqft':[],'agent':[],'gym':[],'transaction_type':[],'price':[]}

In [41]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

# Initialize a dictionary to store the data


for i in tqdm(links, 'scraping'):
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }
    w = requests.get(i, headers=header)
    s = BeautifulSoup(w.content, 'html.parser')

    location = 'NA'
    bed = 'NA'
    bath = 'NA'
    park = 'No covered parking'
    furniture = 'NA'
    transaction = "NA"
    name = re.split(r'/|&', i)[4]

    link = str(i)
    sqft_element = s.find('div', class_='mb-ldp__dtls__body__list')
    sqft = sqft_element.text.strip()[:8] if sqft_element else 'NA'

    price_element = s.find('div', class_='mb-ldp__dtls__price')
    price = price_element.text.lower() if price_element else 'NA'

    agent_element = s.find('span', class_='mb-ldp__contact-info--batch')
    agent = agent_element.text.lower() if agent_element else 'Not certified'

    div = s.find_all(['li', 'div'], class_='mb-ldp__dtls__body__summary--item')
    for element in div:
        text = element.text.lower()
        if 'bed' in text:
            bed = text
        elif 'bath' in text:
            bath = text
        elif 'covered park' in text:
            park = text
        elif 'furnish' in text:
            furniture = text

    location_values = s.find_all('div', class_='mb-ldp__more-dtl__list--value')
    for i in range(len(location_values)):
        if 'Maharashtra' in location_values[i].text:
            location = location_values[i].text.split('type')[0]

    transaction_values = s.find_all('li', class_='mb-ldp__dtls__body__list--item')
    for j in range(len(transaction_values)):
        if 'Transaction' in transaction_values[j].text:
            transaction = transaction_values[j].text.split('type')[1]

    amenities = s.find('div', class_='mb-ldp__lux-amenities')

    if amenities:
        amenities_text = amenities.text.lower()
        gym = 'yes' if 'gymnasium' in amenities_text else 'no'

    # Check if there are any elements matching the selector
    links = s.find_all('a', class_='mb-ldp__dtls__title--link')
    if links:
        # Get the href attribute from the first matching element
        pin = links[0].get('href')
        if pin is None:
            pincode = 'NA'
        else:
            header = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
                'Accept-Language': 'en-US, en;q=0.5'
            }
            w = requests.get(pin, headers=header)
            s = BeautifulSoup(w.content, 'html.parser')

            pincode_element = s.find('div', class_='loc-det__blocks__pincode')
            pincode = pincode_element.text if pincode_element else 'NA'
    else:
        pincode = 'NA'

    d['sqft'].append(sqft)
    d['location'].append(location)
    d['price'].append(price)
    d['agent'].append(agent)
    d['bedrooms'].append(bed)
    d['bathroom'].append(bath)
    d['parking'].append(park)
    d['furniture'].append(furniture)
    d['transaction_type'].append(transaction)
    d['name'].append(name)
    d['gym'].append(gym)
    d['links'].append(link)
    d['pincode'].append(pincode)

# Now, you have collected data in the dictionary 'd'
# You can process or save the data as needed


scraping: 100%|████████████████████████████████████████████████████████████████████████| 50/50 [01:05<00:00,  1.30s/it]


In [9]:
links

['https://www.magicbricks.com/propertyDetails/5-BHK-3100-Sq-ft-Penthouse-FOR-Sale-Bandra-West-in-Mumbai&id=4d423638323831343837',
 'https://www.magicbricks.com/propertyDetails/5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai&id=4d423639323236333639',
 'https://www.magicbricks.com/propertyDetails/4-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Thakur-Village-in-Mumbai&id=4d423637303333353637',
 'https://www.magicbricks.com/propertyDetails/5-BHK-6223-Sq-ft-Penthouse-FOR-Sale-Goregaon-East-in-Mumbai-r1&id=4d423531333232303132',
 'https://www.magicbricks.com/propertyDetails/4-BHK-2800-Sq-ft-Penthouse-FOR-Sale-Akurli-Nagar-in-Mumbai&id=4d423635363938323639',
 'https://www.magicbricks.com/propertyDetails/5-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Worli-Sea-Face-in-Mumbai&id=4d423638313737343231',
 'https://www.magicbricks.com/propertyDetails/3-BHK-4500-Sq-ft-Penthouse-FOR-Sale-Amboli-in-Mumbai&id=4d423636363035383037',
 'https://www.magicbricks.com/propertyDetails/4-BHK-3926-Sq-ft-Penthouse-FOR-Sale-B

In [18]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import concurrent.futures

# Initialize a dictionary to store the data
d = {
    'sqft': [],
    'location': [],
    'pincode': [],
    'price': [],
    'agent': [],
    'bedrooms': [],
    'bathroom': [],
    'parking': [],
    'furniture': [],
    'transaction_type': [],
    'name': [],
    'gym': [],
    'links': []
    
}

# Function to scrape data from a single URL
def scrape_data(url):
    try:
        # Check if the URL is 'nan' or not
        if url == 'nan':
            raise ValueError("Invalid URL: 'nan'")

        header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'
        }
        w = requests.get(url, headers=header)
        s = BeautifulSoup(w.content, 'html.parser')

        location = 'NA'
        bed = 'NA'
        bath = 'NA'
        park = 'No covered parking'
        furniture = 'NA'
        transaction = "NA"
        name = re.split(r'/|&', url)[4]

        link = str(url)
        sqft_element = s.find('div', class_='mb-ldp__dtls__body__list')
        sqft = sqft_element.text.strip()[:8] if sqft_element else 'NA'

        price_element = s.find('div', class_='mb-ldp__dtls__price')
        price = price_element.text.lower() if price_element else 'NA'

        agent_element = s.find('span', class_='mb-ldp__contact-info--batch')
        agent = agent_element.text.lower() if agent_element else 'Not certified'

        div = s.find_all(['li', 'div'], class_='mb-ldp__dtls__body__summary--item')
        for element in div:
            text = element.text.lower()
            if 'bed' in text:
                bed = text
            elif 'bath' in text:
                bath = text
            elif 'covered park' in text:
                park = text
            elif 'furnish' in text:
                furniture = text

        location_values = s.find_all('div', class_='mb-ldp__more-dtl__list--value')
        for i in range(len(location_values)):
            if 'Maharashtra' in location_values[i].text:
                location = location_values[i].text.split('type')[0]

        transaction_values = s.find_all('li', class_='mb-ldp__dtls__body__list--item')
        for j in range(len(transaction_values)):
            if 'Transaction' in transaction_values[j].text:
                transaction = transaction_values[j].text.split('type')[1]

        amenities = s.find('div', class_='mb-ldp__lux-amenities')

        # Initialize 'gym' with a default value
        gym = 'NA'

        if amenities:
            amenities_text = amenities.text.lower()
            gym = 'yes' if 'gymnasium' in amenities_text else 'no'

        # Check if there are any elements matching the selector
        links = s.find_all('a', class_='mb-ldp__dtls__title--link')
        if links:
            # Get the href attribute from the first matching element
            pin = links[0].get('href')
            if pin is None:
                pincode = 'NA'
            else:
                header = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
                    'Accept-Language': 'en-US, en;q=0.5'
                }
                w = requests.get(pin, headers=header)
                s = BeautifulSoup(w.content, 'html.parser')

                pincode_element = s.find('div', class_='loc-det__blocks__pincode')
                pincode = pincode_element.text if pincode_element else 'NA'
        else:
            pincode = 'NA'

        return sqft, location, price, agent, bed, bath, park, furniture, transaction, name, gym, link, pincode

    except Exception as e:
        # Handle any exceptions here (e.g., invalid URLs)
        print(f"Error processing URL '{url}': {str(e)}")
        return None

# Create a thread pool for concurrent scraping
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    results = list(filter(None, tqdm(executor.map(scrape_data, links), total=len(links))))

# Unpack the results and add them to the dictionary 'd'
for result in results:
    d['sqft'].append(result[0])
    d['location'].append(result[1])
    d['price'].append(result[2])
    d['agent'].append(result[3])
    d['bedrooms'].append(result[4])
    d['bathroom'].append(result[5])
    d['parking'].append(result[6])
    d['furniture'].append(result[7])
    d['transaction_type'].append(result[8])
    d['name'].append(result[9])
    d['gym'].append(result[10])
    d['links'].append(result[11])
    d['pincode'].append(result[12])

# Now, you have collected data in the dictionary 'd'
# You can process or save the data as needed


  3%|██▍                                                                         | 449/14220 [01:41<1:02:59,  3.64it/s]

Error processing URL 'nan': Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


  8%|█████▊                                                                     | 1102/14220 [04:03<1:26:08,  2.54it/s]

Error processing URL 'https://www.magicbricks.com/propertyDetails/2-BHK-50-Sq-ft-Residential-House-FOR-Sale-in-Mumbai&id=4d423639333234393331': Exceeded 30 redirects.


 10%|███████▋                                                                   | 1469/14220 [05:20<1:28:02,  2.41it/s]

...

 92%|█████████████████████████████████████████████████████████████████████▋      | 13035/14220 [49:16<03:50,  5.14it/s]

Error processing URL 'nan': Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


 95%|████████████████████████████████████████████████████████████████████████    | 13474/14220 [50:59<03:22,  3.69it/s]

Error processing URL 'nan': Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


100%|████████████████████████████████████████████████████████████████████████████| 14220/14220 [53:52<00:00,  4.40it/s]


In [19]:
pd.DataFrame(d)

,sqft,location,pincode,price,agent,bedrooms,bathroom,parking,furniture,transaction_type,name,gym,links
0,3100sqft,"Park View, Bandra West, Mumbai South West, Maharashtra, Bandra West, Mumbai - Western Mumbai, Maharashtra","Mumbai\nWestern Mumbai, \nBandra West Pin Code: 400050",₹19.0 cr,certified agent,5beds,6baths,4 covered parking,NA,New Property,5-BHK-3100-Sq-ft-Penthouse-FOR-Sale-Bandra-West-in-Mumbai,NA,https://www.magicbricks.com/propertyDetails/5-BHK-3100-Sq-ft-Penthouse-FOR-Sale-Bandra-West-in-Mumbai&id=4d423638323831343837
1,2200sqft,"Piramal Revanta, Behind Nirmal Lifestyle mall, Goregaon Mulund Link Road, Mulund West, Mulund West, Mumbai - Central Mumbai, Maharashtra","Mumbai\nCentral Mumbai, \nMulund West Pin Code: 400080",₹5.90 cr,Not certified,5beds,5baths,3 covered parking,unfurnished,New Property,5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai,yes,https://www.magicbricks.com/propertyDetails/5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai&id=4d423639323236333639
2,3600sqft,"2602 challenger tower, Thakur Village, Mumbai - Western Mumbai, Maharashtra","Mumbai\nWestern Mumbai, Pin Code: 400101",₹12.0 cr,Not certified,4beds,4baths,1 covered parking,NA,Resale,4-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Thakur-Village-in-Mumbai,NA,https://www.magicbricks.com/propertyDetails/4-BHK-4200-Sq-ft-Penthouse-FOR-Sale-Thakur-Village-in-Mumbai&id=4d423637303333353637
3,4114sqft,"Goregaon East, Mumbai, Goregaon East, Mumbai - Western Mumbai, Maharashtra","Mumbai\nWestern Mumbai, \nGoregaon East Pin Code: 400063",₹22.0 cr,Not certified,5beds,6baths,5 covered parking,NA,Resale,5-BHK-6223-Sq-ft-Penthouse-FOR-Sale-Goregaon-East-in-Mumbai-r1,yes,https://www.magicbricks.com/propertyDetails/5-BHK-6223-Sq-ft-Penthouse-FOR-Sale-Goregaon-East-in-Mumbai-r1&id=4d423531333232303132
4,1602sqft,"Kandivali East, Mumbai, Akurli Nagar, Mumbai - Western Mumbai, Maharashtra","Mumbai\nWestern Mumbai, Pin Code: 400101",₹4.95 cr,certified agent,4beds,4baths,3 covered parking,NA,New Property,4-BHK-2800-Sq-ft-Penthouse-FOR-Sale-Akurli-Nagar-in-Mumbai,NA,https://www.magicbricks.com/propertyDetails/4-BHK-2800-Sq-ft-Penthouse-FOR-Sale-Akurli-Nagar-in-Mumbai&id=4d423635363938323639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,2700sqft,"101., Juhu, Mumbai - Western Mumbai, Maharashtra","Mumbai\nWestern Mumbai, \nJuhu Pin Code: 400049",₹15.0 cr,Not certified,4beds,3baths,2 covered parking,furnished,Resale,4-BHK-3100-Sq-ft-Multistorey-Apartment-FOR-Sale-Juhu-in-Mumbai,NA,https://www.magicbricks.com/propertyDetails/4-BHK-3100-Sq-ft-Multistorey-Apartment-FOR-Sale-Juhu-in-Mumbai&id=4d423638353236303239
14208,5615sqft,"Mumbai, Nariman Point, Mumbai - South Mumbai, Maharashtra","Mumbai\nSouth Mumbai, \nNariman Point Pin Code: 400021",₹45.0 cr,Not certified,4beds,5baths,No covered parking,unfurnished,New Property,4-BHK-9065-Sq-ft-Multistorey-Apartment-FOR-Sale-Nariman-Point-in-Mumbai,NA,https://www.magicbricks.com/propertyDetails/4-BHK-9065-Sq-ft-Multistorey-Apartment-FOR-Sale-Nariman-Point-in-Mumbai&id=4d423630313237353533
14209,1926sqft,"L b s road, Mulund West, Mumbai - Central Mumbai, Maharashtra","Mumbai\nCentral Mumbai, \nMulund West Pin Code: 400080",₹6.25 cr,Not certified,4beds,4baths,3 covered parking,NA,Resale,4-BHK-1926-Sq-ft-Multistorey-Apartment-FOR-Sale-Mulund-West-in-Mumbai-r2,NA,https://www.magicbricks.com/propertyDetails/4-BHK-1926-Sq-ft-Multistorey-Apartment-FOR-Sale-Mulund-West-in-Mumbai-r2&id=4d423636353231323333
14210,1578sqft,"Mulund West, Mulund West, Mumbai - Central Mumbai, Maharashtra","Mumbai\nCentral Mumbai, \nMulund West Pin Code: 400080",₹2.95 cr,certified agent,4beds,3baths,1 covered parking,NA,New Property,4-BHK-1578-Sq-ft-Multistorey-Apartment-FOR-Sale-Mulund-West-in-Mumbai-r1,yes,https://www.magicbricks.com/propertyDetails/4-BHK-1578-Sq-ft-Multistorey-Apartment-FOR-Sale-Mulund-West-in-Mumbai-r1&id=4d423430313639373335


In [20]:
g=pd.DataFrame(d)
g.to_csv(r'fianl_data.csv',index=False)

In [35]:
i='https://www.magicbricks.com/propertyDetails/350-Sq-ft-Studio-Apartment-FOR-Sale-Mahavir-Nagar-Kandivali-West-in-Mumbai&id=4d423637343138393231'
header = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36' ,'Accept-Language': 'en-US, en;q=0.5'})
w=requests.get(i,header)
s=BeautifulSoup(w.content , 'html.parser')

In [38]:
for i in range(len(s.find_all('a',class_='mb-ldp__dtls__title--link'))):
    a=(s.find_all('a',class_='mb-ldp__dtls__title--link')[i].get('href'))


https://www.magicbricks.com/Mahavir-Nagar-Kandivali-West-in-Mumbai-Overview
https://www.magicbricks.com/Mahavir-Nagar-Kandivali-West-in-Mumbai-Overview


In [73]:
s.find('div',class_='mb-ldp__lux-amenities').text

'ConvenienceLeisureSportsCafeteria/Food CourtMultipurpose HallMultipurpose CourtsMeditation AreaKids Play Pool With Water SlidesLiftLawn with pathwayCricket net practiceCycling & Jogging TrackIndoor Games RoomKids play areaGymnasiumSwimming PoolClub HouseBadminton courtDownload Brochure'

In [74]:
'gymnasium' in s.find('div',class_='mb-ldp__lux-amenities').text.lower()

True

In [75]:
'Under Construction' or 'Ready to Move' in s.find_all('li', class_='mb-ldp__dtls__body__list--item')[4].text

'Under Construction'

In [83]:
s.find_all('li', class_='mb-ldp__dtls__body__list--item')[i].text

'Transaction typeNew Property'

In [33]:
text=s.find('div',class_='mb-ldp__lux-amenities').text


In [34]:
text
if 'Swimming' in text:
    swimm = 'yes'
if 'Gymnasium' in text:
    gymm = 'yes'

In [35]:
gymm

'yes'

In [36]:
sqft=s.find('div',class_='mb-ldp__dtls__body__list').text.split('sqft')[0]
sqft

'2200'

In [37]:
price=s.find('div',class_='mb-ldp__dtls__price').text
price

'₹5.90 Cr'

In [38]:
s.find_all('div',class_='mb-ldp__more-dtl__list--value')

[<div class="mb-ldp__more-dtl__list--value"><span class="rupees">₹</span>5.9 Cr | <span class="rupees">₹</span>22,000 <span class="mb-ldp__more-dtl__list--value--hint-text">Monthly</span></div>,
 <div class="mb-ldp__more-dtl__list--value"><span class="rupees">₹</span>15.0 Lac</div>,
 <div class="mb-ldp__more-dtl__list--value">P51800005983</div>,
 <div class="mb-ldp__more-dtl__list--value">Piramal Revanta, Behind Nirmal Lifestyle mall, Goregaon Mulund Link Road, Mulund West, Mulund West, Mumbai - Central Mumbai, Maharashtra</div>,
 <div class="mb-ldp__more-dtl__list--value">LBS Marg, Mulund West, Nr Nirmal lifestyle, Mulund, Moti nagar</div>,
 <div class="mb-ldp__more-dtl__list--value">Unfurnished</div>,
 <div class="mb-ldp__more-dtl__list--value">Granite, Vitrified, Wooden</div>]

In [39]:
for i in range(len(s.find_all('li',class_='mb-ldp__more-dtl__list--item'))):
     if 'Maharashtra' in s.find_all('li',class_='mb-ldp__more-dtl__list--item')[i].split():
               print(s.find_all('li',class_='mb-ldp__more-dtl__list--item')[i])
            

TypeError: 'NoneType' object is not callable

In [40]:
for i in range(len(s.findAll('li',class_='mb-ldp__more-dtl__list--item'))):
    if 'Maharashtra' in s.findAll('li',class_='mb-ldp__more-dtl__list--item')[i].text:
        print(s.findAll('li',class_='mb-ldp__more-dtl__list--item')[i].text)
               

AddressPiramal Revanta, Behind Nirmal Lifestyle mall, Goregaon Mulund Link Road, Mulund West, Mulund West, Mumbai - Central Mumbai, Maharashtra


In [19]:
s.findAll('li',class_='mb-ldp__more-dtl__list--item')[6].text

'FlooringGranite, Vitrified, Wooden'

In [24]:
amenities = s.find('div',class_='mb-ldp__lux-amenities')
if amenities!=None:
    for elements in amenities:
        text = elements.text.lower()
        if 'security' in text:
            secure = 'yes'
        if 'gymnasium' in text:
            gym = 'yes'

In [26]:
amenities = s.find('div',class_='mb-ldp__lux-amenities')

In [35]:
amenities.text

'ConvenienceLeisureSportsCafeteria/Food CourtMultipurpose HallMultipurpose CourtsMeditation AreaKids Play Pool With Water SlidesLiftLawn with pathwayCricket net practiceCycling & Jogging TrackIndoor Games RoomKids play areaGymnasiumSwimming PoolClub HouseBadminton courtDownload Brochure'

In [33]:
'security' in amenities.text

False

In [32]:
'Gym' in amenities.text

True

In [11]:
a='https://www.magicbricks.com/propertyDetails/5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai&id=4d423639323236333639'

In [15]:
re.split(r'/|&', a)[4]

'5-BHK-3410-Sq-ft-Penthouse-FOR-Sale-Mulund-West-in-Mumbai'

In [8]:
b

array([ 4,  5,  7, 56,  4])